# 느리게 갱신되는 세그먼트 트리 (Segment Tree With Lazy Propagation)

`-` 참고: https://book.acmicpc.net/ds/segment-tree-lazy-propagation

## 구간 합 구하기 2

- 문제 출처: [백준 10999번](https://www.acmicpc.net/problem/10999)

`-` 느리게 갱신되는 세그먼트 트리 기본 문제이다

`-` 물론 나는 이걸 어떻게 구현하는지 모른다

`-` 단순히 구간 업데이트를 개별로 점 업데이트 하는 건 시간 초과이다

`-` 실제로 출력을 할 때만 값을 알면 되니 그 전까지 구간 업데이트를 모아둔 뒤 출력 쿼리 때 계산하자

`-` 이를 위해 임의의 노드에 대해 구간 업데이트로 해당 구간에 누적된 값을 나타내는 lazy를 도입하자

`-` 구간 업데이트에서 해당 구간을 담당하는 노드의 lazy 값을 갱신하자

`-` 예컨대 전체 구간에 $d$를 더한다면 루트 노드의 lazy 값에 $d$를 더해주면 된다

`-` 그런데 단순히 lazy 값만 갱신하면 문제가 생긴다

`-` 예컨대 루트 노드의 오른쪽 자식이 담당하는 구간에 $d$를 더한다고 해보자

`-` 그럼 $\operatorname{lazy}[3] = \operatorname{lazy}[3] + d$가 된다

`-` 이때 전체 구간의 합을 계산해보자

`-` 이는 $\operatorname{tree}[1]$인데 $\operatorname{tree}[1]$은 이전과 변화가 없으니 틀린 결과를 도출한다

`-` 임의의 노드가 담당하는 구간을 $(s,e)$라 하고 갱신할 구간을 $(l,r)$이라 할 때 $l \le s \le e \le r$이면 해당 노드의 lazy 값에 반영하면 된다

`-` 그렇지 않고 일부 구간만 겹칠 땐 tree 배열에 겹친 부분의 구간 합을 누적한 뒤 자식 노드에 $d$만큼 구간 업데이트 된다는 걸 전파하자

`-` 이제 구간 합을 계산하는 쿼리를 처리해야 한다

`-` 기본적으로 $l \le s \le e \le r$이면 해당 노드가 저장하고 있는 구간 합에 lazy 값과 구간의 길이를 곱한 값을 더해서 반환하면 된다

`-` 이제 일부 구간이 겹친다고 해보자

`-` 나는 [수열과 쿼리 21](https://www.acmicpc.net/problem/16975) 문제에서 해결한 것처럼 $\operatorname{aggregate}$ 변수를 도입했다

`-` 그 후 자식 노드를 기준으로 겹친 구간만큼 lazy 값을 계산해 $\operatorname{aggregate}$에 누적하여 전파했다

`-` 그런데 해당 부분을 계속 틀려서 chatgpt의 도움을 조금 받았다

`-` 구간 합 쿼리를 실행했으므로 여태까지 누적된 lazy 값을 tree에 반영해주자

`-` 그리고 lazy 값을 자식들에게 전파한 뒤 자기 자신의 lazy 값을 $0$으로 초기화하면 된다

`-` 구간 합이므로 자식들의 쿼리 결과의 합을 반환하면 마무리된다

`-` 느리게 갱신되는 세그먼트 트리의 시간 복잡도는 일반적인 세그먼트 트리의 시간 복잡도와 동일하다

`-` 따라서 전체 알고리즘의 시간 복잡도는 $O((M+K)\log N)$이다

In [47]:
import math


def init_tree(array, tree, node, start, end):
    if start == end:
        tree[node] = array[start]
        return
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    init_tree(array, tree, left_child, start, mid)
    init_tree(array, tree, right_child, mid + 1, end)
    tree[node] = tree[left_child] + tree[right_child]


def update_tree(left, right, value, tree, lazy, node, start, end):
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        lazy[node] += value
        return value * (end - start + 1)
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    left_result = update_tree(left, right, value, tree, lazy, left_child, start, mid)
    right_result = update_tree(left, right, value, tree, lazy, right_child, mid + 1, end)
    tree[node] += left_result + right_result
    return left_result + right_result


def range_sum_query(left, right, tree, lazy, node, start, end):
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        return tree[node] + lazy[node] * (end - start + 1)
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    tree[node] += lazy[node] * (end - start + 1)
    lazy[left_child] += lazy[node]
    lazy[right_child] += lazy[node]
    lazy[node] = 0
    left_sum = range_sum_query(left, right, tree, lazy, left_child, start, mid)
    right_sum = range_sum_query(left, right, tree, lazy, right_child, mid + 1, end)
    return left_sum + right_sum


def solution():
    N, M, K = map(int, input().split())
    array = [int(input()) for _ in range(N)]
    h = math.ceil(math.log2(N))
    tree_size = 2**(h + 1) - 1
    tree = [0] * (tree_size + 1)
    lazy = [0] * (tree_size + 1)
    start, end = 0, N - 1
    root = 1
    init_tree(array, tree, root, start, end)
    for _ in range(M + K):
        query = list(map(int, input().split()))
        a = query[0]
        if a == 1:
            b, c, d = query[1:]
            update_tree(b - 1, c - 1, d, tree, lazy, root, start, end)
        else:
            b, c = query[1:]
            range_sum = range_sum_query(b - 1, c - 1, tree, lazy, root, start, end)
            print(range_sum)


solution()

# input
# 5 2 2
# 1
# 2
# 3
# 4
# 5
# 1 3 4 6
# 2 2 5
# 1 1 3 -2
# 2 2 5

 5 2 2
 1
 2
 3
 4
 5
 1 3 4 6
 2 2 5


26


 1 1 3 -2
 2 2 5


22


`-` 문제 풀었으니까 공부하고 오자